<a href="https://colab.research.google.com/github/jbenasuli/final_project/blob/database/dev/database/Amazon_Vine-PySpark-ETLs/PRELIM_ETL_Apparel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.1.1'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bi

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-05-03 04:02:02--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.09s   

2021-05-03 04:02:02 (10.9 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
import datetime
from pyspark.sql.functions import to_date
from pyspark.sql.functions import col
from pyspark.sql.types import StructField, StringType, IntegerType, StructType, BooleanType, DateType
from pyspark import SparkFiles

### Load Amazon Data into Spark DataFrame

Note: Enter URL for Desired Dataset

In [5]:
#1 load product segment
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Apparel_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|      24485154|Easy Tool Stainle...|         Apparel|          4|            0|          0|   N|                Y|★ THESE REALLY DO...|These Really Do W...| 2013-01-14|
|         US|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|     363128556|V28 Women Cowl Ne...|         Apparel|          5|    

### Create DataFrame - Perform Preliminary Cleaning

In [6]:
#1 Check the schema, print row & column count
df.printSchema()
print((df.count(), len(df.columns)))

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)

(5906333, 15)


In [7]:
#2 Drop the round 1 columns
columns_to_drop = ['marketplace', 'product_parent', 'vine', 'review_headline', 'review_body', 'review_date']
df_dropped = df.drop(*columns_to_drop)
df_dropped.show()

+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+-----------------+
|customer_id|     review_id|product_id|       product_title|product_category|star_rating|helpful_votes|total_votes|verified_purchase|
+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+-----------------+
|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|Easy Tool Stainle...|         Apparel|          4|            0|          0|                Y|
|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|V28 Women Cowl Ne...|         Apparel|          5|            1|          2|                Y|
|   12608825| RWQEDYAX373I1|B01I497BGY|James Fiallo Men'...|         Apparel|          5|            0|          0|                Y|
|   25482800|R231YI7R4GPF6J|B01HDXFZK6|Belfry Gangster 1...|         Apparel|          5|            0|          0|                Y|
|    9310286|R3KO3W45DD0L1K|B01G6MBEBY|JAEDEN Women's Be...|  

In [8]:
#3 Filter Step 1:
df_filtered = df_dropped.filter(df_dropped.verified_purchase == 'Y')
df_filtered.show
print((df_filtered.count(), len(df_filtered.columns)))

(5312774, 9)


In [9]:
#4 drop filtered verified_purchase column
columns_to_drop = ['verified_purchase']
df_dropped_2 = df_filtered.drop(*columns_to_drop)
df_dropped_2.show()

+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+
|customer_id|     review_id|product_id|       product_title|product_category|star_rating|helpful_votes|total_votes|
+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+
|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|Easy Tool Stainle...|         Apparel|          4|            0|          0|
|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|V28 Women Cowl Ne...|         Apparel|          5|            1|          2|
|   12608825| RWQEDYAX373I1|B01I497BGY|James Fiallo Men'...|         Apparel|          5|            0|          0|
|   25482800|R231YI7R4GPF6J|B01HDXFZK6|Belfry Gangster 1...|         Apparel|          5|            0|          0|
|    9310286|R3KO3W45DD0L1K|B01G6MBEBY|JAEDEN Women's Be...|         Apparel|          5|            0|          0|
|   26631939|R1C4QH63NFL5NJ|B01FWRXN0Y|Levi's Boys' 514 ...|         App

## Create Analysis-Specific DFs/Tables
### Perform Analysis-Specific Transforms

### Segmentation Analysis DF

In [10]:
# 1 Create Segmentation DF by Droppig Addtional Columns
segmentation_cols_drop = ['review_id', 'product_id', 'product_title', 'star_rating', 'helpful_votes', 'total_votes']
segmentation_dropped_df = df_dropped_2.drop(*segmentation_cols_drop)
segmentation_dropped_df.show()

+-----------+----------------+
|customer_id|product_category|
+-----------+----------------+
|   32158956|         Apparel|
|    2714559|         Apparel|
|   12608825|         Apparel|
|   25482800|         Apparel|
|    9310286|         Apparel|
|   26631939|         Apparel|
|   48785098|         Apparel|
|   39548589|         Apparel|
|   29355866|         Apparel|
|   27477484|         Apparel|
|   17685865|         Apparel|
|   19787539|         Apparel|
|   44196725|         Apparel|
|   45510794|         Apparel|
|   42025740|         Apparel|
|   27097189|         Apparel|
|   13808904|         Apparel|
|   30225894|         Apparel|
|   45223824|         Apparel|
|   22300783|         Apparel|
+-----------+----------------+
only showing top 20 rows



Note: must change Category Label name in withColumnRenamed('count(product_category)', 'name')

In [11]:
#2 Segmentation GroupBy
#2a GroupBy customer_id
#2b Count product_category and rename count columns as Segment Name
segment_df = segmentation_dropped_df.groupby("customer_id")\
.agg({'product_category':'count'}).withColumnRenamed('count(product_category)', 'apparel')
#2c Check results
segment_df.show()

+-----------+-------+
|customer_id|apparel|
+-----------+-------+
|   13780696|      1|
|   43380682|      1|
|   27446622|      1|
|     120706|      3|
|   15915947|      2|
|   37520920|      1|
|   14309190|      8|
|   22316505|      1|
|   43949054|      2|
|   21102747|      1|
|   19021463|      1|
|   16075459|      7|
|   51475782|      2|
|   35471383|      3|
|   34788104|      2|
|    1169723|      1|
|   40659946|      1|
|    1319956|      1|
|     193166|      2|
|   44514239|      1|
+-----------+-------+
only showing top 20 rows



In [12]:
#3 Check segment_df Schema and Row Count
segment_df.printSchema()
print(segment_df.count())

root
 |-- customer_id: integer (nullable = true)
 |-- apparel: long (nullable = false)

2958291


Note: Column Name in df.sort('name' ...) must align with Column name from step 2

In [13]:
#4 Filter for Top n Results
#4a Declare number of rows to filter by (100,000)
row_count = 100000
#4a Sort by Segment Desc and limit to row_count
filtered_segment_df = segment_df.sort('apparel', ascending=False).limit(row_count)
#4b Check Results
filtered_segment_df.show()
print(filtered_segment_df.count())

+-----------+-------+
|customer_id|apparel|
+-----------+-------+
|   37474421|    258|
|   45825821|    249|
|   15006109|    223|
|   10558864|    192|
|    6724212|    175|
|   22985631|    165|
|   45187592|    162|
|   14230771|    154|
|   19672530|    152|
|   25388273|    148|
|   37965911|    146|
|   35178127|    143|
|   38166052|    143|
|    7809052|    140|
|   40333751|    140|
|   17580988|    133|
|   42995420|    131|
|    6534413|    127|
|    2174967|    124|
|   43310494|    124|
+-----------+-------+
only showing top 20 rows

100000


### Segmentation ETL Complete - Add Database Export Code

In [14]:
# Configure settings for RDS
mode = "append"
jdbc_url='jdbc:postgresql://<connection_srtring>'
config = {"user":'<username>', 
          "password": '<password>', 
          "driver":"org.postgresql.Driver"}

Note: table name in table='name_segment' must align with table name in Postgres

In [15]:
# Write segment table to Postgres/RDS
# 1 min 5 secs
filtered_segment_df.write.jdbc(url=jdbc_url, table='apparel_segment', mode=mode, properties=config)

## Apriori Analysis DF
### Enter Apriori Transfomations Below - Use df_dropped_2 as Start Point

In [16]:
# 1 Create Apriori DF by First Dropping Addtional Columns
apriori_cols_drop = ['product_category', 'product_title', 'star_rating', 'helpful_votes', 'total_votes']
apriori_dropped_df = df_dropped_2.drop(*apriori_cols_drop)
apriori_dropped_df.show()

+-----------+--------------+----------+
|customer_id|     review_id|product_id|
+-----------+--------------+----------+
|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|
|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|
|   12608825| RWQEDYAX373I1|B01I497BGY|
|   25482800|R231YI7R4GPF6J|B01HDXFZK6|
|    9310286|R3KO3W45DD0L1K|B01G6MBEBY|
|   26631939|R1C4QH63NFL5NJ|B01FWRXN0Y|
|   48785098|R2GP65O1U9N7BP|B01EXNH1HE|
|   39548589|R3O29CT5MQQ3XQ|B01E7OL09O|
|   29355866|R1ZECD2AA8QFF6|B01DXHX81O|
|   27477484|R2S79GCF6J89OA|B01DDULIJK|
|   17685865|R1CBCUGNP37MPK|B01BOKOL4A|
|   19787539|R3NU4WUR5ZNV1V|B01B3Q4Q0O|
|   44196725|R32EPCJ3XF8LGE|B01ADDSL9U|
|   45510794|R1XIBC6WQBW31M|B019P1X5XI|
|   42025740|R1P11MVDZ65LMH|B019MDXIXG|
|   27097189|R1OJA3DJL0VDDK|B019438FEG|
|   13808904|R1THWA5YRJLOOF|B0178HGNIA|
|   30225894| R738LCNRSJVXP|B016VIU0QI|
|   45223824|R1N3Z13931J3O9|B016PUU3VO|
|   22300783|R1LBNTP7E8N89Z|B016AQNDM4|
+-----------+--------------+----------+
only showing top 20 rows



### Prelim Apriori ETL Complete - Add Database Export Code

In [17]:
# Configure settings for RDS
mode = "append"
jdbc_url='jdbc:postgresql://<connection_srtring>'
config = {"user":'<username>', 
          "password": '<password>', 
          "driver":"org.postgresql.Driver"}

Note: table name in table='segment_apriori' must align with table name in Postgres

In [18]:
# Write segment_apriori table to RDS
# 13 mins 15 secs
apriori_dropped_df.write.jdbc(url=jdbc_url, table='apparel_apriori', mode=mode, properties=config)

## Amazon Reviews S3 -> Postgres/RDS ETL Complete



### Run Queries in Postgres to Confirm Segment ETL
Check Row Count of Segment Table - SELECT COUNT(*) FROM apparel_segment;

Check 10 Rows of Segment Table - SELECT * FROM apparel_segment LIMIT(10);


In [19]:
# Upon Confirmation of Above Checks Run This Cell
print('Segment ETL Successful')

Segment ETL Successful


### Run Queries in Postgres to Confirm Apriori ETL
Check Row Count of Apriori Table - SELECT COUNT(*) FROM apparel_apriori;

Check 10 Rows of Apriori Table - SELECT * FROM apparel_apriori LIMIT(10);

In [20]:
# Upon Confirmation of Above Checks Run This Cell
print('Apriori ETL Successful')

Apriori ETL Successful
